# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f787085bbe0>
├── 'a' --> tensor([[ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920]])
└── 'x' --> <FastTreeValue 0x7f787085ba60>
    └── 'c' --> tensor([[ 1.3590,  0.6017, -1.7111,  1.3381],
                        [ 0.3507, -1.1927,  1.8651,  0.6519],
                        [-1.0781, -0.9777,  1.1607, -0.9982]])

In [4]:
t.a

tensor([[ 0.4285, -0.6385, -0.9169],
        [-0.1129,  0.7228, -0.2920]])

In [5]:
%timeit t.a

67.3 ns ± 1.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f787085bbe0>
├── 'a' --> tensor([[-1.2857, -0.5086,  0.3192],
│                   [-0.8909,  1.6290,  0.6049]])
└── 'x' --> <FastTreeValue 0x7f787085ba60>
    └── 'c' --> tensor([[ 1.3590,  0.6017, -1.7111,  1.3381],
                        [ 0.3507, -1.1927,  1.8651,  0.6519],
                        [-1.0781, -0.9777,  1.1607, -0.9982]])

In [7]:
%timeit t.a = new_value

73.5 ns ± 1.16 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920]]),
    x: Batch(
           c: tensor([[ 1.3590,  0.6017, -1.7111,  1.3381],
                      [ 0.3507, -1.1927,  1.8651,  0.6519],
                      [-1.0781, -0.9777,  1.1607, -0.9982]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4285, -0.6385, -0.9169],
        [-0.1129,  0.7228, -0.2920]])

In [11]:
%timeit b.a

69.7 ns ± 0.958 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.4461,  1.6867,  1.5129],
               [ 0.3874, -0.3588,  0.5670]]),
    x: Batch(
           c: tensor([[ 1.3590,  0.6017, -1.7111,  1.3381],
                      [ 0.3507, -1.1927,  1.8651,  0.6519],
                      [-1.0781, -0.9777,  1.1607, -0.9982]]),
       ),
)

In [13]:
%timeit b.a = new_value

595 ns ± 16.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

968 ns ± 13.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.9 µs ± 183 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

281 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

273 µs ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f78708625b0>
├── 'a' --> tensor([[[ 0.4285, -0.6385, -0.9169],
│                    [-0.1129,  0.7228, -0.2920]],
│           
│                   [[ 0.4285, -0.6385, -0.9169],
│                    [-0.1129,  0.7228, -0.2920]],
│           
│                   [[ 0.4285, -0.6385, -0.9169],
│                    [-0.1129,  0.7228, -0.2920]],
│           
│                   [[ 0.4285, -0.6385, -0.9169],
│                    [-0.1129,  0.7228, -0.2920]],
│           
│                   [[ 0.4285, -0.6385, -0.9169],
│                    [-0.1129,  0.7228, -0.2920]],
│           
│                   [[ 0.4285, -0.6385, -0.9169],
│                    [-0.1129,  0.7228, -0.2920]],
│           
│                   [[ 0.4285, -0.6385, -0.9169],
│                    [-0.1129,  0.7228, -0.2920]],
│           
│                   [[ 0.4285, -0.6385, -0.9169],
│                    [-0.1129,  0.7228, -0.2920]]])
└── 'x' --> <FastTreeValue 0x7f7870862fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42.8 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f77c350a790>
├── 'a' --> tensor([[ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920],
│                   [ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920],
│                   [ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920],
│                   [ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920],
│                   [ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920],
│                   [ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920],
│                   [ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920],
│                   [ 0.4285, -0.6385, -0.9169],
│                   [-0.1129,  0.7228, -0.2920]])
└── 'x' --> <FastTreeValue 0x7f7847fb2970>
    └── 'c' --> tensor([[ 1.3590,  0.6017, -1.7111,  1.3381],
                        [ 0.3507, -1.1927,  1.8651,  0.6519],
                 

In [23]:
%timeit t_cat(trees)

39.9 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.8 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4285, -0.6385, -0.9169],
                [-0.1129,  0.7228, -0.2920]],
       
               [[ 0.4285, -0.6385, -0.9169],
                [-0.1129,  0.7228, -0.2920]],
       
               [[ 0.4285, -0.6385, -0.9169],
                [-0.1129,  0.7228, -0.2920]],
       
               [[ 0.4285, -0.6385, -0.9169],
                [-0.1129,  0.7228, -0.2920]],
       
               [[ 0.4285, -0.6385, -0.9169],
                [-0.1129,  0.7228, -0.2920]],
       
               [[ 0.4285, -0.6385, -0.9169],
                [-0.1129,  0.7228, -0.2920]],
       
               [[ 0.4285, -0.6385, -0.9169],
                [-0.1129,  0.7228, -0.2920]],
       
               [[ 0.4285, -0.6385, -0.9169],
                [-0.1129,  0.7228, -0.2920]]]),
    x: Batch(
           c: tensor([[[ 1.3590,  0.6017, -1.7111,  1.3381],
                       [ 0.3507, -1.1927,  1.8651,  0.6519],
                       [-1.0781, -0.9777,  1.1607, -0.9982]],
         

In [26]:
%timeit Batch.stack(batches)

99.1 µs ± 3.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920],
               [ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920],
               [ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920],
               [ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920],
               [ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920],
               [ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920],
               [ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920],
               [ 0.4285, -0.6385, -0.9169],
               [-0.1129,  0.7228, -0.2920]]),
    x: Batch(
           c: tensor([[ 1.3590,  0.6017, -1.7111,  1.3381],
                      [ 0.3507, -1.1927,  1.8651,  0.6519],
                      [-1.0781, -0.9777,  1.1607, -0.9982],
                      [ 1.3590,  0.6017, -1.7111,  1.3381],
                      [ 0.3507, -

In [28]:
%timeit Batch.cat(batches)

176 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

556 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
